In [1]:
import tensorflow as tf
import keras
import transformers
print("TensorFlow version:", tf.__version__)
print("Transformers version:", transformers.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.17.1
Transformers version: 4.47.0
Keras version: 3.5.0


In [2]:

import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import torch
from torch.utils.data import Dataset

In [3]:
# Load datasets
train_df = pd.read_csv('/kaggle/input/malaylam/AWM_train.csv')
dev_df = pd.read_csv('/kaggle/input/malaylam/AWM_dev.csv')
# test_without_label = pd.read_csv('/kaggle/input/malaylam/AWM_test_without_labels.csv')
test_with_label = pd.read_csv('/kaggle/input/testlabel-data/AWM_test_with_labels (1).csv')

In [4]:
print(train_df.shape)
print(dev_df.shape)
print(test_with_label.shape)

(2933, 2)
(629, 2)
(629, 3)


In [5]:
train_df.head(10)

,Text,Class
0,നവ്യയുടെ കയ്യിന്ന് കിട്ടിയതാണല്ലോ. ഇവൾക്ക് അതൊ...,Abusive
1,"""ഇവരുടെ പ്രശ്നം അസൂയ ആണ്, സുന്ദരികൾ ആയ നടിമാരോ...",Abusive
2,ചുമ്മാതല്ല ഇവളെ പണ്ട് ലവർ അലക്കി വിട്ടത്...വായ...,Abusive
3,"""ഒരു സിനിമയിൽ ജഗതി മദാമ്മയായിട്ട് വരുന്നുണ്ടല്...",Non-Abusive
4,ഈ വർഷത്തെ ബൂലോക തോൽവി പരാജയം അതിനുള്ള ഓസ്‌ക്കാ...,Abusive
5,ബാക്കിൽ നിക്കുന്ന ചേച്ചി : എന്ത് വെറുപ്പിക്കൽ ...,Abusive
6,പ്രയാഗ ശരിക്കും അമേരിക്കൻ അമ്മായി ലുക്ക് ആയല...,Non-Abusive
7,പ്രയാഗ ആർന്നോ ഞാൻ വിചാരിച്ചു ഏതോ ഇംഗ്ലീഷ് കാരി...,Non-Abusive
8,ശെരിക്കും ഇതിന്റെ പാട്ട് വട്ടാണ് വട്ടാണ് എനിക...,Non-Abusive
9,നല്ല ഒരു നായിയായിരുന്നു പക്ഷേ ഇപ്പോ ആരും അഭിനയ...,Abusive


In [6]:
dev_df.head(10)

,Text,Class
0,ഞാനും എന്റെ ഫാമിലി മോളെ വ്യക്തിത്വവും ഗെയിം കണ...,Non-Abusive
1,സാരമില്ല മോളെ നീ ആരുടെ മുന്നിലും കരയരുത് അത് ...,Non-Abusive
2,ഡോക്ടർക്ക് ഇങ്ങനെ തന്നെ വേണം. ഈ വിവരമില്ലാത്ത...,Abusive
3,"""ട്രോളന്മാർക്കും, യൂട്യൂബ് ചാനൽസ് നും ചാകര ആയി .""",Non-Abusive
4,ഇപ്പോ പറഞ്ഞ നോ നേര്തെ പറഞ്ഞഗിൽ ഇത്തരം മോശം കമന...,Non-Abusive
5,നിന്റെ ചേച്ചിമാർ ഓസിന് വോട്ട് പിടിച്ചപ്പോൾ ഓർക...,Non-Abusive
6,നൈസ് ആയി ഒരു കാരണത്തിനു കാത്ത് നിൽക്കുക ആയിരുന...,Abusive
7,അന്തം ഫാൻസിനു ഇനി പിരിഞ്ഞു പോവാം,Non-Abusive
8,"""ഡോക്ടർ ഇത്രെയും ചീപ്പ്‌ ആയി പോയല്ലോ,,, ഇവന് ന...",Non-Abusive
9,നല്ല രീതിയിൽ തീരേണ്ട ഒരു കാര്യമായിരുന്നു. ഇപ്പ...,Non-Abusive


In [7]:
test_with_label.head(10)

,id,Text,Class
0,1,സൂരജ് നിന്റെ ആര് ആണ് ആള് ഇറക്കി പേടിപ്പിക്കുക ആണോ,Non-Abusive
1,2,എത്ര അലക്കി വെളുപ്പിച്ചാലും നിനക്കു അർഹത ഇല്ലാ...,Abusive
2,3,50 ലക്ഷം കയ്യിൽ വയ്ക്കാൻ ഒരിക്കലും യോഗ്യത ഇല്ല...,Non-Abusive
3,4,"""ബിഗ് ബോസ്സിൽ നിങ്ങളുടെ അഭിനയം എന്തായിരുന്നു,മ...",Abusive
4,5,അത് അങ്ങനെയാ നമ്മുടെ ഉള്ളിൽ നന്മ ഉണ്ടെങ്കിൽ പട...,Non-Abusive
5,6,ഇനി കൂടുതൽ ഒന്നും പറയാതിരിക്കുന്നതായിരിക്കും ന...,Non-Abusive
6,7,നിങ്ങളെ ആരും ഇഷ്ടപ്പെടുന്നില്ലാ നിങ്ങളുടെ കാര്...,Abusive
7,8,ഈ ഞാൻ ഞാൻ ...... ഞാനെന്താണെന്ന് മനസ്സിലായി ......,Abusive
8,9,ഇതിൻ്റെ ട്രോൾ കാണാൻ കേറി വരൂ,Non-Abusive
9,10,പണത്തിന്റെ കാര്യം വന്നപ്പോൾ ദേഷ്യം ഒക്കെ പോയോ,Non-Abusive


In [8]:
import re
from bs4 import BeautifulSoup
def remove_punctuations(text):
    punctuations = '''!()-[]{};:'"“\’,<>./?@#$%^&*_~—॥'''
    return ''.join([char for char in text if char not in punctuations])

# Function to replace unwanted strings (URLs, emojis, HTML tags, etc.)
def replace_strings(text):
    # Remove URLs
    url_pattern = r'https?://\S+|www\.\S+'
    text = re.sub(url_pattern, ' ', text)

    # Remove emojis
    emoji_pattern = re.compile("[" 
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"  
        u"\u2000-\u206F"          # general punctuations
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r' ', text)

    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Normalize spaces
    text = re.sub(r'\s+', ' ', text)

    return text

# Function for full preprocessing
def preprocessing(text):
    cleaned_text = remove_punctuations(replace_strings(text))
    return cleaned_text.lower()


In [9]:
# Apply preprocessing to all datasets
train_df['cleanText'] = train_df['Text'].apply(lambda x: preprocessing(str(x)))
dev_df['cleanText'] = dev_df['Text'].apply(lambda x: preprocessing(str(x)))
test_with_label['cleanText'] = test_with_label['Text'].apply(lambda x: preprocessing(str(x)))


<ipython-input-8-e98d2e6d74aa>:26: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()
<ipython-input-8-e98d2e6d74aa>:26: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


In [10]:
# Combine all text for vocabulary analysis
train_corpus = train_df['cleanText'].sum()
dev_corpus = dev_df['cleanText'].sum()
test_corpus = test_with_label['cleanText'].sum()

In [11]:
# Vocabulary and OOV analysis
train_vocab = set(train_corpus.split())
dev_vocab = set(dev_corpus.split())
test_vocab = set(test_corpus.split())

oov_dev = dev_vocab - train_vocab
oov_test = test_vocab - train_vocab

print(f"Number of unique words in training data: {len(train_vocab)}")
print(f"Number of unique words in development data: {len(dev_vocab)}")
print(f"Number of unique words in test data: {len(test_vocab)}")
print(f"OOV words in dev dataset: {len(oov_dev)}")
print(f"OOV words in test dataset: {len(oov_test)}")

Number of unique words in training data: 15675
Number of unique words in development data: 4647
Number of unique words in test data: 4586
OOV words in dev dataset: 2525
OOV words in test dataset: 2496


In [15]:
# Label encoding for train, dev, and test datasets
train_df['enc_label'] = train_df['Class'].replace({'Abusive': 1, 'Non-Abusive': 0})
dev_df['enc_label'] = dev_df['Class'].replace({'Abusive': 1, 'Non-Abusive': 0})
test_with_label['enc_label'] = test_with_label['Class'].replace({'Abusive': 1, 'Non-Abusive': 0})

print("Training dataset label counts:")
print(train_df['enc_label'].value_counts())
print("Development dataset label counts:")
print(dev_df['enc_label'].value_counts())
print("Test dataset label counts:")
print(test_with_label['enc_label'].value_counts())


Training dataset label counts:
enc_label
1    1531
0    1402
Name: count, dtype: int64
Development dataset label counts:
enc_label
0    326
1    303
Name: count, dtype: int64
Test dataset label counts:
enc_label
1    323
0    306
Name: count, dtype: int64


<ipython-input-15-f9d929a275b1>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['enc_label'] = train_df['Class'].replace({'Abusive': 1, 'Non-Abusive': 0})
<ipython-input-15-f9d929a275b1>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_df['enc_label'] = dev_df['Class'].replace({'Abusive': 1, 'Non-Abusive': 0})
<ipython-input-15-f9d929a275b1>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['cleanText'])
X_test_tfidf = tfidf_vectorizer.transform(test_with_label['cleanText'])

y_train = train_df['enc_label']
y_test = test_with_label['enc_label']

# Individual models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Naive Bayes": MultinomialNB(),
    "SVM": SVC(kernel='linear', C=1, probability=True),  # Enable probability for ensemble
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
}

# Train and evaluate individual models
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train_tfidf, y_train)
    
    # Evaluation on Test Set
    y_test_pred = model.predict(X_test_tfidf)
    print(f"Classification Report for Test Set ({name}):")
    print(classification_report(y_test, y_test_pred))
    print("\n")

Training Logistic Regression...
Classification Report for Test Set (Logistic Regression):
              precision    recall  f1-score   support

           0       0.62      0.66      0.64       306
           1       0.66      0.61      0.63       323

    accuracy                           0.64       629
   macro avg       0.64      0.64      0.64       629
weighted avg       0.64      0.64      0.64       629



Training Naive Bayes...
Classification Report for Test Set (Naive Bayes):
              precision    recall  f1-score   support

           0       0.63      0.67      0.65       306
           1       0.67      0.63      0.65       323

    accuracy                           0.65       629
   macro avg       0.65      0.65      0.65       629
weighted avg       0.65      0.65      0.65       629



Training SVM...
Classification Report for Test Set (SVM):
              precision    recall  f1-score   support

           0       0.63      0.68      0.65       306
           

In [21]:
# Ensemble technique: Majority Voting
print("Training Ensemble Model with Majority Voting...")

# Create an ensemble with Logistic Regression, Naive Bayes, and SVM
ensemble_model = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(max_iter=1000, random_state=42)),
        ('nb', MultinomialNB()),
        ('svm', SVC(kernel='linear', C=1, probability=True))
    ],
    voting='soft'  # soft voting for better probabilities
)

ensemble_model.fit(X_train_tfidf, y_train)
y_test_ensemble_pred = ensemble_model.predict(X_test_tfidf)

# Classification report for the ensemble model
print("Classification Report for Test Set (Ensemble Model - Majority Voting):")
print(classification_report(y_test, y_test_ensemble_pred))

Training Ensemble Model with Majority Voting...
Classification Report for Test Set (Ensemble Model - Majority Voting):
              precision    recall  f1-score   support

           0       0.63      0.67      0.65       306
           1       0.67      0.62      0.64       323

    accuracy                           0.65       629
   macro avg       0.65      0.65      0.65       629
weighted avg       0.65      0.65      0.65       629

